In [1]:
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib
import zipfile
import os
import pandas as pd
import warnings
import rarfile, csv
import torch.nn as nn
import torch
import torch.optim as optim
import torch.utils.data as data_utils
from scipy.stats import pearsonr
import seaborn as sns
from sklearn.model_selection import train_test_split
from random import shuffle
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import random
import torch.nn.functional as F
import sklearn
import skorch
import pickle
import math
from collections import Counter
from fastprogress import master_bar, progress_bar
from sklearn.utils import resample





from skorch.utils import params_for
from torch.autograd import Variable




warnings.filterwarnings(action='once')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


%matplotlib inline

### RNN classifier


In [13]:
import time
import math

epoch_num = 400
learning_rate = 0.00001
#dropout_p = 0.5
n_features = len(dataset_encoded_normalized.columns)-1
use_cuda = True
#weight_decay = 0.0001
BATCH_SIZE = 50
L2 = 0.001

In [14]:
class ANN(nn.Module):

    def __init__(self, n_features, output_size):

        super(ANN, self).__init__()
        
        self.fc1 = nn.Linear(n_features, 64).cuda()
        self.fc2 = nn.Linear(64, 128).cuda()
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 256).cuda()
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(256, 128).cuda()
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(128, 32).cuda()
        self.relu5 = nn.ReLU()
        self.fc6 = nn.Linear(32, output_size).cuda()
        self.softmax = nn.LogSoftmax(dim=1).cuda()



    def one_step_forward(self, input):

        out = self.fc1(input)
        out = self.relu2(self.fc2(out))
        out = self.relu3(self.fc3(out))
        out = self.relu4(self.fc4(out))
        out = self.relu5(self.fc5(out))
        prob = self.fc6(out)
        
        softmax = self.softmax(prob)

        return softmax
        
        
        

In [15]:
class Trainer:
    
    def __init__(self, optimizer = torch.optim.Adam, learning_rate = learning_rate, L2 = L2,
                                                    n_features = n_features,
                                                    output_size = output_size,
                                                    #dropout_p = dropout_p,
                                                    criterion = nn.NLLLoss()):
        
        self.module = ANN(n_features = n_features, output_size = output_size)
        self.n_features = n_features
        self.criterion = criterion
        
        self.optimizer_adam = optimizer(self.module.parameters(), lr = learning_rate, weight_decay = L2)
        
        
    def train_step(self, Xi, yi):

        loss = None
        accuracy = None
                
        self.module.zero_grad()
                    
        output = self.module.one_step_forward(torch.tensor(Xi).type(torch.FloatTensor).to(device=device))        
            
        probabilities = torch.exp(output)
        
        _, predicted = torch.max(output.data, 1)
        
        if  len(yi) != 0:
            
            correct = (predicted.squeeze(0) == torch.tensor(yi).type(torch.LongTensor).to(device=device)).sum()
            accuracy = correct
            loss = self.get_loss(output, yi)
        
            if self.module.training:

                loss.backward()
                self.optimizer_adam.step()

        return {'loss':  loss, 'probabilities': probabilities, 'acc': accuracy}

    def infer(self, Xi, yi=[]):

        self.module.eval()
        output = self.train_step(Xi, yi)
        self.module.train()
        
        return output
    
    def get_loss(self, ys_pred, y_true):
        
        return self.criterion(ys_pred, torch.tensor(y_true).type(torch.LongTensor).to(device = device))

In [ ]:
fault_predictor = Trainer()
mb = master_bar(range(1, epoch_num))
mb2 = master_bar(range(1, epoch_num))
mb.names = ['Training Error', 'Validation Error']
mb2.names = ['Training acc', 'Validation acc']


train_batch_num = round(len(X_train.to_numpy()) / BATCH_SIZE)
test_batch_num = round(len(X_test.to_numpy()) / BATCH_SIZE)


epoch_tra_acc = {}
epoch_tra_error = {}

epoch_evl_acc = {}
epoch_evl_error = {}

for i in mb:
    

    X_train_batch = batch_maker(X_train.to_numpy(), BATCH_SIZE)
    y_train_batch = batch_maker(y_train.to_numpy(), BATCH_SIZE)
    x_test_batch = batch_maker(X_test.to_numpy(), BATCH_SIZE)
    y_test_batch = batch_maker(y_test.to_numpy(), BATCH_SIZE)
    
    tra_acc = []
    tra_error = []
    
    evl_acc = []
    evl_error = []
    
    
    for j in progress_bar(range(train_batch_num), parent=mb):
        
        mb.child.comment = f'training'
        
        X_tra = next(X_train_batch)
        y_tra = next(y_train_batch)
        stat = fault_predictor.train_step(X_tra, y_tra)
        tra_acc.append((stat['acc'].cpu().detach().numpy())/len(X_tra))
        tra_error.append((stat['loss'].cpu().detach().numpy())/len(X_tra))
        #print(tra_error[-1])
        #print(tra_acc[-1])
        #break
        
    mb.write("### Training ### Epoch:{} loss:{} acc:{}".format(i , round(np.sum(tra_error)/(len(tra_error)*1.), 5), round(np.sum(tra_acc)/(len(tra_acc)*1.), 5)))
    epoch_tra_error[i] = round(np.sum(tra_error)/(len(tra_error)*1.), 5)
    epoch_tra_acc[i] = (round(np.sum(tra_acc)/(len(tra_acc)*1.), 5))
    #graphs_acc = [[epoch_tra_error.keys(), epoch_tra_error.values()],[epoch_evl_error.keys(), epoch_evl_error.values()]]
    graphs_acc = [[epoch_tra_error.keys(), epoch_tra_error.values()],[epoch_evl_error.keys(), epoch_evl_error.values()]]
    graphs_acc2 = [[epoch_tra_acc.keys(), epoch_tra_acc.values()],[epoch_evl_acc.keys(), epoch_evl_acc.values()]]
    mb.update_graph(graphs_acc)
    mb2.update_graph(graphs_acc2)

    
    if (i % 2) == 0:

        for k in progress_bar(range(test_batch_num), parent=mb):
            
            mb.child.comment = f'validation'
            
            X_tes = next(x_test_batch)
            y_tes = next(y_test_batch)
            
            stat = fault_predictor.infer(X_tes, y_tes)
            evl_acc.append((stat['acc'].cpu().detach().numpy())/len(X_tes)) 
            evl_error.append((stat['loss'].cpu().detach().numpy())/len(X_tes))
        mb.write("### Validation ### Epoch:{} loss:{} acc:{}".format(i , round(np.sum(evl_error)/(len(evl_error)*1.), 5), (round(np.sum(evl_acc)/(len(evl_acc)*1.), 5))))
        
        epoch_evl_acc[i] = (round(np.sum(evl_acc)/(len(evl_acc)*1.), 5))
        epoch_evl_error[i] = round(np.sum(evl_error)/(len(evl_error)*1.), 5)
        
        #graphs_acc = [[epoch_tra_error.keys(), epoch_tra_error.values()], [epoch_evl_error.keys(), epoch_evl_error.values()]]
        graphs_acc = [[epoch_tra_error.keys(), epoch_tra_error.values()],[epoch_evl_error.keys(), epoch_evl_error.values()]]
        graphs_acc2 = [[epoch_tra_acc.keys(), epoch_tra_acc.values()],[epoch_evl_acc.keys(), epoch_evl_acc.values()]]
        mb.update_graph(graphs_acc)
        mb2.update_graph(graphs_acc2)

In [22]:
pred = fault_predictor.infer(torch.tensor(X_test.to_numpy()).type(torch.FloatTensor).to(device=device))

c:\users\administrator\pytorch\lib\site-packages\ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [23]:
_, predicted = torch.max(pred['probabilities'], 1)

In [ ]:
cr = classification_report(y_test, predicted.cpu().detach().numpy())
print(cr)

In [25]:
def vis_conf_mat(conf_arr):
    norm_conf = []
    for i in conf_arr:
        a = 0
        tmp_arr = []
        a = sum(i, 0)
        for j in i:
            tmp_arr.append(float(j)/float(a))
        norm_conf.append(tmp_arr)

    fig = plt.figure(figsize=(8, 6))
    plt.clf()
    ax = fig.add_subplot(111)
    ax.set_aspect(1)
    res = ax.imshow(np.array(norm_conf), cmap=plt.cm.jet, 
                    interpolation='nearest')

    width, height = conf_arr.shape

    for x in range(width):
        for y in range(height):
            ax.annotate(str(conf_arr[x][y]), xy=(y, x), 
                        horizontalalignment='center',
                        verticalalignment='center')

    cb = fig.colorbar(res)
    alphabet = list(PROM_CAUSE.keys())
    plt.xticks(range(width), alphabet[:width],rotation='vertical')
    plt.yticks(range(height), alphabet[:height])
    plt.savefig('confusion_matrix.png', format='png')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  


conf_arr = confusion_matrix(y_test.astype(int), predicted.cpu().detach().numpy())

vis_conf_mat(conf_arr)